In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tqdm
!pip install requests


In [ ]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from PIL import Image
from io import BytesIO


In [ ]:
# path to your CSV
csv_path = "/kaggle/input/mlhack/train.csv"
csv_path2="/kaggle/input/mlhack/test.csv"
# read CSV
df_train = pd.read_csv(csv_path)
df_test = pd.read_csv(csv_path2)

print(df_train.shape)
print(df_test.shape)


**DOWNLOAD TRAIN IMAGES**

In [ ]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
from pathlib import Path
import time

# ============== CONFIG ======================
train_csv_path = "/kaggle/input/mlhack/train.csv"
train_image_folder = "/kaggle/working/train_images"
IMAGE_COLUMN = "image_link"
MAX_WORKERS = min(64, cpu_count())  # Adjust concurrency
# ============================================

# Create directories
os.makedirs(train_image_folder, exist_ok=True)

# Load CSVs
train_df = pd.read_csv(train_csv_path)

print("✅ Columns in train:", train_df.columns.tolist())

# Extract image links
train_links = train_df[IMAGE_COLUMN].dropna().unique().tolist()

print(f"Train images to download: {len(train_links)}")


# ======================================================
# 🧩 Function to download a single image robustly
# ======================================================
def download_image(image_url, save_folder):
    try:
        filename = Path(image_url).name
        save_path = os.path.join(save_folder, filename)

        # Skip if already downloaded
        if os.path.exists(save_path):
            return "exists"

        headers = {"User-Agent": "Mozilla/5.0"}  # Prevent blocking
        response = requests.get(image_url, headers=headers, timeout=10)

        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            return "ok"
        else:
            return f"error_{response.status_code}"

    except Exception as e:
        return f"fail_{str(e)[:40]}"


# ======================================================
# 🧠 Parallel Downloader
# ======================================================
def parallel_download(image_links, folder):
    print(f"📥 Starting downloads → {folder}")
    start = time.time()
    os.makedirs(folder, exist_ok=True)
    
    download_fn = partial(download_image, save_folder=folder)
    
    results = []
    with Pool(MAX_WORKERS) as pool:
        for result in tqdm(pool.imap_unordered(download_fn, image_links), total=len(image_links)):
            results.append(result)
    
    ok = sum(1 for r in results if r == "ok" or r == "exists")
    fail = len(results) - ok
    print(f"✅ Completed: {ok} | ⚠️ Failed: {fail} | ⏱️ Time: {round(time.time() - start, 2)}s")
    return ok, fail


# ======================================================
# 🚀 Run Train + Test Downloads
# ======================================================
ok_train, fail_train = parallel_download(train_links, train_image_folder)

print("\n📊 FINAL STATS:")
print(f"Train → Downloaded: {ok_train}, Failed: {fail_train}")

# Optional sanity check
print(f"\n🖼️ Train images in folder: {len(os.listdir(train_image_folder))}")


**TEXT EMBEDDINGS AND FEATRUE ENGINEERING**

In [ ]:
# Install required libs (run once)
!pip install -q transformers datasets sentencepiece

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q --upgrade pip
!pip install -q transformers accelerate torch torchvision --no-cache-dir

*mapping downloaded train images ti thier coressponding text(catalog_content) and url*

In [ ]:
import pandas as pd
import os

# Load your dataset
train_df = pd.read_csv("/kaggle/input/mlhack/train.csv")

# Extract only the useful columns
train_df = train_df[["image_link", "catalog_content"]]

# Derive image filenames from URLs (if needed)
train_df["image_name"] = train_df["image_link"].apply(lambda x: os.path.basename(x))

# Keep only rows where the image actually exists
image_folder = "/kaggle/working/train_images"
train_df["exists"] = train_df["image_name"].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))
train_df = train_df[train_df["exists"] == True].reset_index(drop=True)

print("✅ Final usable samples:", len(train_df))


**->FEATURE ENGINEERING**

In [ ]:

train_csv_path= "/kaggle/input/mlhack/train.csv"
test_csv_path = "/kaggle/input/mlhack/test.csv"
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

In [ ]:
import re
import pandas as pd

def extract_weight(text):
    match = re.search(r'(\d+(?:\.\d+)?)\s*(g|kg|oz|lb|ml|l)', text, re.IGNORECASE)
    if match:
        value, unit = match.groups()
        value = float(value)
        unit = unit.lower()
        if unit == 'kg': value *= 1000
        elif unit == 'lb': value *= 453.592
        elif unit == 'oz': value *= 28.3495
        elif unit == 'l': value *= 1000  # ml
        return value
    return 0
train_df['weight_g'] = train_df['catalog_content'].apply(extract_weight)

test_df['weight_g'] = test_df['catalog_content'].apply(extract_weight)


In [ ]:
def extract_pack_count(text):
    match = re.search(r'(\d+)\s*(?:x|X|pack of)\s*\d*', text, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 1

train_df['pack_count'] = train_df['catalog_content'].apply(extract_pack_count)
test_df['pack_count'] = test_df['catalog_content'].apply(extract_pack_count)


In [ ]:


train_df['num_tokens'] = train_df['catalog_content'].str.split().str.len()
train_df['num_digits'] = train_df['catalog_content'].str.count(r'\d')
train_df['num_uppercase_words'] = train_df['catalog_content'].str.count(r'\b[A-Z]{2,}\b')

test_df['num_tokens'] = test_df['catalog_content'].str.split().str.len()
test_df['num_digits'] = test_df['catalog_content'].str.count(r'\d')
test_df['num_uppercase_words'] = test_df['catalog_content'].str.count(r'\b[A-Z]{2,}\b')



In [ ]:
# -------------------------
# Utility preprocessing functions
# -------------------------
def light_clean(text):
    """Minimal cleaning for DistilBERT: remove HTML, collapse whitespace, keep punctuation."""
    if text is None:
        return ""
    text = str(text)
    # remove html tags
    text = re.sub(r'<[^>]+>', ' ', text)
    # replace newlines / tabs with space
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    # collapse multi-spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example regex extraction for weight/quantity (optional but high ROI)
def extract_quantity_and_weight(text):
    """
    Returns tuple (quantity_int_or_nan, weight_grams_or_nan)
    This is a heuristic extractor — tweak regexes for your data.
    """
    q = np.nan
    w = np.nan
    if not text:
        return q, w
    # pack counts like 'pack of 6', '6 pack', '6 x 250g'
    m = re.search(r'(\bpack of\b|\bpack\b|\bx\b)\s*(\d{1,3})', text, flags=re.I)
    if m:
        try:
            q = int(re.search(r'(\d{1,3})', m.group(0)).group(1))
        except:
            q = np.nan
    # weights like '250 g', '250g', '0.5 kg', '12 oz'
    m2 = re.search(r'(\d+(?:\.\d+)?)\s*(kg|g|gram|grams|oz|ounce|ounces|ml|l|litre|liter)\b', text, flags=re.I)
    if m2:
        val = float(m2.group(1))
        unit = m2.group(2).lower()
        # convert to grams/ml where possible
        if unit in ['kg']:
            w = val * 1000.0
        elif unit in ['g','gram','grams']:
            w = val
        elif unit in ['mg']:
            w = val / 1000.0
        elif unit in ['l','litre','liter']:
            w = val * 1000.0  # liters -> ml (approx)
        elif unit in ['ml']:
            w = val
        elif unit in ['oz','ounce','ounces']:
            w = val * 28.3495
    return q, w


In [ ]:
# -------------------------
# Load dataset(s)
# -------------------------
print("Loading CSVs...")
train_df = pd.read_csv(TRAIN_CSV)
print("Train rows:", len(train_df))
if os.path.exists(TEST_CSV):
    test_df = pd.read_csv(TEST_CSV)
    print("Test rows:", len(test_df))
else:
    test_df = None
print("Using text column:", TEXT_COL)

# Fill missing and clean text
train_df[TEXT_COL] = train_df[TEXT_COL].fillna("").astype(str).apply(light_clean)
if test_df is not None:
    test_df[TEXT_COL] = test_df[TEXT_COL].fillna("").astype(str).apply(light_clean)

# Optional: extract numeric structured features from the text (quantity, weight)
print("Extracting quantity/weight heuristics (optional)...")
train_qw = train_df[TEXT_COL].apply(extract_quantity_and_weight)
train_df['qty'] = [x[0] for x in train_qw]
train_df['weight_g'] = [x[1] for x in train_qw]

if test_df is not None:
    test_qw = test_df[TEXT_COL].apply(extract_quantity_and_weight)
    test_df['qty'] = [x[0] for x in test_qw]
    test_df['weight_g'] = [x[1] for x in test_qw]


*->TEST EMBEDDINGS AND FEATURE EMBEDDINGS OF TEST SET*

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder

# ===============================
# Config
# ===============================
TEXT_COL = "catalog_content"
BATCH_SIZE = 64
MAX_LEN = 128
MODEL_NAME = "distilbert-base-uncased"
OUTPUT_DIR = "/kaggle/working/test_embeddings_features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TEST_CSV = "/kaggle/input/mlhack/test.csv"

# ===============================
# Load test dataset
# ===============================
test_df = pd.read_csv(TEST_CSV)

# Clean text
def light_clean(text):
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r'<[^>]+>', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_df[TEXT_COL] = test_df[TEXT_COL].fillna("").astype(str).apply(light_clean)

# ===============================
# Device and model setup
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# -------------------------------
# Mean pooling
def mean_pooling(token_embeddings, attention_mask):
    mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeds = torch.sum(token_embeddings * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    return sum_embeds / sum_mask

# -------------------------------
# Encode test texts to embeddings
def encode_texts_to_embeddings(df, text_col, out_path, batch_size=BATCH_SIZE, max_length=MAX_LEN):
    n = len(df)
    emb_dim = model.config.hidden_size
    embeddings = np.zeros((n, emb_dim), dtype=np.float32)

    texts = df[text_col].fillna("").astype(str).tolist()
    with torch.no_grad():
        for start in tqdm(range(0, n, batch_size), desc=f"Encoding {os.path.basename(out_path)}"):
            end = min(n, start + batch_size)
            batch_texts = texts[start:end]
            encoded = tokenizer(batch_texts, padding=True, truncation=True,
                                max_length=max_length, return_tensors='pt')
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pooled = mean_pooling(outputs.last_hidden_state, attention_mask).cpu().numpy()
            embeddings[start:end, :] = pooled

    embeddings = np.nan_to_num(embeddings, nan=0.0)
    np.save(out_path, embeddings)
    print(f"Saved test text embeddings: {out_path}")
    return embeddings

# -------------------------------
# Save engineered numeric/categorical features for test
def save_engineered_features(df, out_path, text_col=TEXT_COL):
    exclude_cols = [text_col, 'sample_id']
    numeric_df = df[[c for c in df.columns if c not in exclude_cols]].copy()

    for col in numeric_df.select_dtypes(include='object').columns:
        numeric_df[col] = numeric_df[col].fillna("nan_missing")
        le = LabelEncoder()
        numeric_df[col] = le.fit_transform(numeric_df[col].astype(str))

    numeric_df = numeric_df.fillna(0)
    np.save(out_path, numeric_df.values.astype(np.float32))
    print(f"Saved test engineered numeric features: {out_path}")
    return numeric_df.values

# ===============================
# Paths for saving test embeddings/features
test_text_emb_path = os.path.join(OUTPUT_DIR, "test_catalog_text_emb.npy")
test_features_path = os.path.join(OUTPUT_DIR, "test_engineered_numeric_features.npy")

# ===============================
# Process test set
test_text_emb = encode_texts_to_embeddings(test_df, TEXT_COL, test_text_emb_path)
test_features = save_engineered_features(test_df, test_features_path, text_col=TEXT_COL)

print("✅ Test embeddings and features saved.")
print("Test text embeddings:", test_text_emb_path)
print("Test engineered numeric features:", test_features_path)


*->TRAIN EMBEDDINGS AND FEATURE ENGG EMBEDDINGS OF TRAIN SET*

In [ ]:
# ===============================
# DistilBERT embeddings + Engineered numeric features (NaN-safe) for train/test
# ===============================
import os
import re
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder

# ===============================
# Config
# ===============================
TEXT_COL = "catalog_content"  # Text column
BATCH_SIZE = 64
MAX_LEN = 128
MODEL_NAME = "distilbert-base-uncased"
OUTPUT_DIR = "/kaggle/working/embeddings_features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# CSV paths
TRAIN_CSV = "/kaggle/input/mlhack/train.csv"
TEST_CSV = "/kaggle/input/mlhack/test.csv"

# ===============================
# Load datasets
# ===============================
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

# Fill missing text with empty string and clean
def light_clean(text):
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r'<[^>]+>', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df[TEXT_COL] = train_df[TEXT_COL].fillna("").astype(str).apply(light_clean)
test_df[TEXT_COL] = test_df[TEXT_COL].fillna("").astype(str).apply(light_clean)

# ===============================
# Device and model setup
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# -------------------------------
# Mean pooling
def mean_pooling(token_embeddings, attention_mask):
    mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeds = torch.sum(token_embeddings * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    return sum_embeds / sum_mask

# -------------------------------
# Encode text to embeddings
def encode_texts_to_embeddings(df, text_col, out_path, batch_size=BATCH_SIZE, max_length=MAX_LEN):
    n = len(df)
    emb_dim = model.config.hidden_size
    embeddings = np.zeros((n, emb_dim), dtype=np.float32)

    texts = df[text_col].fillna("").astype(str).tolist()
    with torch.no_grad():
        for start in tqdm(range(0, n, batch_size), desc=f"Encoding {os.path.basename(out_path)}"):
            end = min(n, start + batch_size)
            batch_texts = texts[start:end]
            encoded = tokenizer(batch_texts, padding=True, truncation=True,
                                max_length=max_length, return_tensors='pt')
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pooled = mean_pooling(outputs.last_hidden_state, attention_mask).cpu().numpy()
            embeddings[start:end, :] = pooled

    # Replace any NaNs (just in case)
    embeddings = np.nan_to_num(embeddings, nan=0.0)
    np.save(out_path, embeddings)
    print(f"Saved text embeddings: {out_path}")
    return embeddings

# -------------------------------
# Save engineered numeric/categorical features
def save_engineered_features(df, out_path, text_col=TEXT_COL, target_col=None, train_le_dict=None):
    # Exclude text and optionally target
    exclude_cols = [text_col, 'sample_id']
    if target_col is not None and target_col in df.columns:
        exclude_cols.append(target_col)

    engineered_cols = [c for c in df.columns if c not in exclude_cols]
    numeric_df = df[engineered_cols].copy()

    le_dict = {} if train_le_dict is None else train_le_dict

    for col in numeric_df.select_dtypes(include='object').columns:
        if train_le_dict is None:  # train set
            le = LabelEncoder()
            numeric_df[col] = numeric_df[col].fillna("nan_missing")
            numeric_df[col] = le.fit_transform(numeric_df[col].astype(str))
            le_dict[col] = le
        else:  # test set
            le = le_dict[col]
            numeric_df[col] = numeric_df[col].fillna("nan_missing")
            numeric_df[col] = numeric_df[col].apply(lambda x: x if x in le.classes_ else "nan_missing")
            numeric_df[col] = le.transform(numeric_df[col].astype(str))

    # Fill numeric NaNs with 0
    numeric_df = numeric_df.fillna(0)

    np.save(out_path, numeric_df.values.astype(np.float32))
    print(f"Saved engineered numeric features: {out_path}")
    return numeric_df.values, le_dict

# ===============================
# Paths for saving
train_text_emb_path = os.path.join(OUTPUT_DIR, "train_catalog_text_emb.npy")
train_features_path = os.path.join(OUTPUT_DIR, "train_engineered_numeric_features.npy")
test_text_emb_path = os.path.join(OUTPUT_DIR, "test_catalog_text_emb.npy")
test_features_path = os.path.join(OUTPUT_DIR, "test_engineered_numeric_features.npy")

# ===============================
# Process train
train_text_emb = encode_texts_to_embeddings(train_df, TEXT_COL, train_text_emb_path)
train_features, le_dict = save_engineered_features(train_df, train_features_path, text_col=TEXT_COL, target_col='price')

# ===============================
# Process test (no target column in test)
test_text_emb = encode_texts_to_embeddings(test_df, TEXT_COL, test_text_emb_path)
test_features, _ = save_engineered_features(test_df, test_features_path, text_col=TEXT_COL, target_col=None, train_le_dict=le_dict)

# ===============================
print("✅ All embeddings and features saved separately for train and test.")
print("Train text embeddings:", train_text_emb_path)
print("Train engineered numeric features:", train_features_path)
print("Test text embeddings:", test_text_emb_path)
print("Test engineered numeric features:", test_features_path)


**LOADING TEXT EMBED, FEATURE EMBED AND IMAGE EMBEDS OF TRAIN AND TEST**

In [ ]:
import numpy as np
import pandas as pd

# Load your train dataframe and embeddings
train_df = pd.read_csv("/kaggle/input/mlhack/train.csv")
train_text_embeds = np.load("/kaggle/input/embeddings/train_catalog_text_emb.npy")
image_embeds = np.load("/kaggle/input/embeddings/image_embeddings.npy")
test_fe_embd=np.load("/kaggle/input/embeddings/train_engineered_numeric_features (1).npy")
print("Text embeddings:", train_text_embeds.shape)
print("Image embeddings:", image_embeds.shape)
test_fe_embd.shape

*fill missing image embed vector with a similar embed image vector from train_image_embed*

In [ ]:
import numpy as np

# Your arrays
text_embeds = train_text_embeds
img_embeds = image_embeds
num_feat = test_fe_embd

# Check and fix
if img_embeds.shape[0] < text_embeds.shape[0]:
    missing_rows = text_embeds.shape[0] - img_embeds.shape[0]
    print(f"⚠️ Missing {missing_rows} image embedding row(s). Fixing using mean vector...")

    # Compute mean image embedding vector
    mean_vector = np.mean(img_embeds, axis=0, keepdims=True)

    # Append missing rows (duplicate mean_vector)
    img_embeds_fixed = np.vstack([img_embeds, np.repeat(mean_vector, missing_rows, axis=0)])
else:
    img_embeds_fixed = img_embeds

print("✅ Fixed image embedding shape:", img_embeds_fixed.shape)

# Sanity check
assert text_embeds.shape[0] == img_embeds_fixed.shape[0] == num_feat.shape[0]
print("✅ All arrays aligned:", text_embeds.shape, img_embeds_fixed.shape, num_feat.shape)


In [ ]:
img_embeds_fixed.shape

In [ ]:
import numpy as np
import pandas as pd

# Load your train dataframe and embeddings
test_df = pd.read_csv("/kaggle/input/mlhack/test.csv")
test_text_embeds = np.load("/kaggle/input/embeddings/test_catalog_text_emb (2).npy")
image_embeds_test = np.load("/kaggle/working/image_test_embeddings.npy")
test_feat_embd=np.load("/kaggle/input/embeddings/test_engineered_numeric_features (1).npy")
print("Text embeddings:", test_text_embeds.shape)
print("Image embeddings:", image_embeds_test.shape)
test_feat_embd.shape

*fill missing image embed vector with a similar embed image vector from test_image_embed*

In [ ]:
import numpy as np

# Your arrays
text_embeds_test = test_text_embeds
img_embeds_test = image_embeds_test
num_feat_test= test_feat_embd

# Check and fix
if img_embeds.shape[0] < text_embeds.shape[0]:
    missing_rows = text_embeds.shape[0] - img_embeds.shape[0]
    print(f"⚠️ Missing {missing_rows} image embedding row(s). Fixing using mean vector...")

    # Compute mean image embedding vector
    mean_vector = np.mean(img_embeds, axis=0, keepdims=True)

    # Append missing rows (duplicate mean_vector)
    img_embeds_fixed_test = np.vstack([img_embeds, np.repeat(mean_vector, missing_rows, axis=0)])
else:
    img_embeds_fixed_test= img_embeds

print("✅ Fixed image embedding shape:", img_embeds_fixed_test.shape)

# Sanity check
assert text_embeds.shape[0] == img_embeds_fixed_test.shape[0] == num_feat.shape[0]
print("✅ All arrays aligned:", text_embeds_test.shape, img_embeds_fixed_test.shape, num_feat_test.shape)


In [ ]:
test_feat_embd

In [ ]:
train_df.head()

In [ ]:
np.isnan(test_feat_embd).any(), np.isnan(test_feat_embd).sum()


In [ ]:
# -------------------------
# Concatenate all TRAIN features
# -------------------------
X_train_full = np.concatenate([text_embeds, img_embeds_fixed, num_feat], axis=1)

print("Final train shape:", X_train_full.shape)

In [ ]:
# -------------------------
# Concatenate all TEST features
# -------------------------
X_test_full = np.concatenate([text_embeds_test,  img_embeds_fixed_test , num_feat_test], axis=1)

print("Final train shape:", X_test_full.shape)

In [ ]:
np.isnan(X_train_full).any(), np.isnan(X_train_full).sum()


In [ ]:
import numpy as np
train_df = pd.read_csv("/kaggle/input/mlhack/train.csv")

# Apply log1p (i.e., log(1 + x)) to avoid log(0)
y = np.log1p(train_df["price"].values)

print("Before log transform:", train_df["price"].describe())
print("\nAfter log transform:", pd.Series(y).describe())


In [ ]:
from sklearn.model_selection import train_test_split
# -------------------------
# Train/validation split
# -------------------------
X_train_split, X_valide, y_train_split, y_valide = train_test_split(
    X_train_full, y, test_size=0.2, random_state=42
)


In [ ]:
X_train_split.shape

In [ ]:
import numpy as np

# y_true and y_pred are original scale values
def smape(y_true, y_pred):
    return 100 * np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred)) / 2))

**TRAIN SET IMAGE EMBEDDINGS USING RESNET**

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, image_folder, transform):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image_name"]
        img_path = os.path.join(self.image_folder, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
            image = self.transform(image)
        except Exception as e:
            # fallback: black image if something fails
            image = torch.zeros(3, 224, 224)

        return image


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # remove final layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)
resnet.eval()

print("✅ Using device:", device)


In [ ]:
image_folder = "/kaggle/working/train_images"

dataset = ImageDataset(train_df, image_folder, image_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
all_features = []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Extracting image embeddings (batched)"):
        batch = batch.to(device)
        features = resnet(batch)                      # shape: [B, 2048, 1, 1]
        features = features.view(features.size(0), -1)  # shape: [B, 2048]
        all_features.append(features.cpu().numpy())

# Combine all batches
image_embeddings = np.concatenate(all_features, axis=0)
print("✅ Image embeddings shape:", image_embeddings.shape)


In [ ]:
np.save("/kaggle/working/image_embeddings.npy", image_embeddings)
print("✅ Saved image embeddings successfully!")


**TEST IMAGE DOWNLOAD AND CREATE TEST IMAGE EMBEDS USING RESNET**

In [ ]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
from pathlib import Path
import time

# ============== CONFIG ======================
train_csv_path = "/kaggle/input/mlhack/test.csv"
train_image_folder = "/kaggle/working/test_images"
IMAGE_COLUMN = "image_link"
MAX_WORKERS = min(64, cpu_count())  # Adjust concurrency
# ============================================

# Create directories
os.makedirs(train_image_folder, exist_ok=True)

# Load CSVs
train_df = pd.read_csv(train_csv_path)

print("✅ Columns in train:", train_df.columns.tolist())

# Extract image links
train_links = train_df[IMAGE_COLUMN].dropna().unique().tolist()

print(f"Train images to download: {len(train_links)}")


# ======================================================
# 🧩 Function to download a single image robustly
# ======================================================
def download_image(image_url, save_folder):
    try:
        filename = Path(image_url).name
        save_path = os.path.join(save_folder, filename)

        # Skip if already downloaded
        if os.path.exists(save_path):
            return "exists"

        headers = {"User-Agent": "Mozilla/5.0"}  # Prevent blocking
        response = requests.get(image_url, headers=headers, timeout=10)

        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            return "ok"
        else:
            return f"error_{response.status_code}"

    except Exception as e:
        return f"fail_{str(e)[:40]}"


# ======================================================
# 🧠 Parallel Downloader
# ======================================================
def parallel_download(image_links, folder):
    print(f"📥 Starting downloads → {folder}")
    start = time.time()
    os.makedirs(folder, exist_ok=True)
    
    download_fn = partial(download_image, save_folder=folder)
    
    results = []
    with Pool(MAX_WORKERS) as pool:
        for result in tqdm(pool.imap_unordered(download_fn, image_links), total=len(image_links)):
            results.append(result)
    
    ok = sum(1 for r in results if r == "ok" or r == "exists")
    fail = len(results) - ok
    print(f"✅ Completed: {ok} | ⚠️ Failed: {fail} | ⏱️ Time: {round(time.time() - start, 2)}s")
    return ok, fail


# ======================================================
# 🚀 Run Train + Test Downloads
# ======================================================
ok_train, fail_train = parallel_download(train_links, train_image_folder)

print("\n📊 FINAL STATS:")
print(f"Train → Downloaded: {ok_train}, Failed: {fail_train}")

# Optional sanity check
print(f"\n🖼️ Train images in folder: {len(os.listdir(train_image_folder))}")

*MAP DOWNLOADED IMAGES TO ITS CORESSPONDING TEXT AND URL*

In [ ]:
import pandas as pd
import os

# Load your dataset
train_df = pd.read_csv("/kaggle/input/mlhack/test.csv")

# Extract only the useful columns
train_df = train_df[["image_link", "catalog_content"]]

# Derive image filenames from URLs (if needed)
train_df["image_name"] = train_df["image_link"].apply(lambda x: os.path.basename(x))

# Keep only rows where the image actually exists
image_folder = "/kaggle/working/test_images"
train_df["exists"] = train_df["image_name"].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))
train_df = train_df[train_df["exists"] == True].reset_index(drop=True)
print("✅ Final usable samples:", len(train_df))


In [ ]:
from torchvision import transforms

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, image_folder, transform):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image_name"]
        img_path = os.path.join(self.image_folder, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
            image = self.transform(image)
        except Exception as e:
            # fallback: black image if something fails
            image = torch.zeros(3, 224, 224)

        return image


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # remove final layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)
resnet.eval()

print("✅ Using device:", device)


In [ ]:
image_folder = "/kaggle/working/test_images"

dataset = ImageDataset(train_df, image_folder, image_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
all_features = []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Extracting image embeddings (batched)"):
        batch = batch.to(device)
        features = resnet(batch)                      # shape: [B, 2048, 1, 1]
        features = features.view(features.size(0), -1)  # shape: [B, 2048]
        all_features.append(features.cpu().numpy())

# Combine all batches
image_embeddings = np.concatenate(all_features, axis=0)
print("✅ Image embeddings shape:", image_embeddings.shape)


In [ ]:
np.save("/kaggle/working/image_test_embeddings.npy", image_embeddings)
print("✅ Saved image embeddings successfully!")


In [ ]:
import time

start = time.time()
from xgboost import XGBRegressor

model = XGBRegressor(
    tree_method="gpu_hist",
    predictor="gpu_predictor",
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror",  # ✅ regression objective
    eval_metric="rmse"              # ✅ root mean square error
)


model.fit(X_train_split, y_train_split, verbose=True)
print("✅ Model trained in", round(time.time() - start, 2), "seconds")


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

y_pred = model.predict(X_valide)

mse = mean_squared_error(y_valide, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_valide, y_pred)
r2 = r2_score(y_valide, y_pred)
smape_score = smape(y_valide, y_pred)

print("SMAPE:", smape_score, "%")
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2 Score:", r2)


In [ ]:
import pandas as pd
import numpy as np

# Load test CSV
TEST_CSV = "/kaggle/input/mlhack/test.csv"
test_df = pd.read_csv(TEST_CSV)

# Predict with both models
xgb_test_preds = xgb_model.predict(X_test_full)
mlp_test_preds = mlp_model(torch.tensor(X_test_full, dtype=torch.float32).to(device)).cpu().detach().numpy().flatten()

# Ensemble
final_test_preds = 0.6 * xgb_test_preds + 0.4 * mlp_test_preds

# Convert back from log
final_test_preds_exp = np.expm1(final_test_preds)

# Create submission DataFrame
submission_df = pd.DataFrame({
    "sample_id": test_df["sample_id"].iloc[:X_test_full.shape[0]],  # align sample_id with available rows
    "price": final_test_preds_exp.flatten()
})

# Save CSV
submission_df.to_csv("/kaggle/working/XGBoost_mlp_predictions.csv", index=False)
print("✅ Predictions saved to tabnet_test_predictions.csv")
